In [ ]:
!pip install face-recognition

In [ ]:
!test -f lfw.tgz || wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
!tar -xf lfw.tgz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import face_recognition.api as face_recognition
import numpy as np

In [ ]:
import glob

In [ ]:
files = {}

for f in glob.glob('/content/drive/MyDrive/senya/data/*'):
  nam = f.rsplit('/', 1)[1].split('@', 1)[0]
  files[nam] = f

In [ ]:
faces = {}

for k, v in files.items():
  pic = face_recognition.load_image_file(v)
  loc = face_recognition.face_locations(pic, number_of_times_to_upsample=0, model='cnn')
  print(f'{k} loaded, got {len(loc)} faces')
  if len(loc) == 1:
    enc = face_recognition.face_encodings(pic, known_face_locations=loc)
    faces[k] = enc

In [ ]:

def extract_faces(im, mode='RGB', model='cnn', upsample=0):
    im0=im
    if not isinstance(im, PIL.Image.Image):
        im = PIL.Image.open(im)
    im = np.array(im.convert(mode))

    loc = face_recognition.face_locations(im, number_of_times_to_upsample=upsample, model=model)
    if len(loc) == 0:
        return []

    enc = face_recognition.face_encodings(im, known_face_locations=loc)
    return enc

In [ ]:
faces2 = {}

for k, v in files.items():
  enc = extract_faces(v)
  if len(enc) == 1:
    faces2[k] = enc

In [ ]:
facediff2 = {k: cmp_face(supersenya, f) for k, f in faces2.items()}
sorted(facediff2.items(), key=itemgetter(1), reverse=True)

In [ ]:
def cmp_face(v1, v2):
  if isinstance(v2, list) and len(v2) > 1:
    raise Exception('v2 cant be list')
  if isinstance(v1, list) and len(v1) > 1:
    # return min(np.linalg.norm(x - v2) for x in v1)
    return np.mean([np.linalg.norm(x - v2) for x in v1])
  return np.linalg.norm(v1 - v2)

def same_face(v1, v2, acc=0.6):
  return cmp_face(v1, v2) <= acc

In [ ]:
supersenya = [faces[k][0] for k in (
    'photo_1',
    'photo_24',
    'photo_6',
    'photo_21',
    'photo_22',
    # 'photo_17',
)]

In [ ]:
supersenya = [faces[k][0] for k in (
    'photo_1',
    'photo_24',
    'photo_6',
    'photo_21',
    'photo_22',
    # 'photo_17',
)]

In [ ]:
import io

In [ ]:
from numpy.lib.npyio import _savez as savez_internal

In [ ]:
savez_internal('drive/MyDrive/senya/supersenya.npz', [], {'supersenya': np.array(supersenya)}, compress=True)

In [ ]:
f = io.BytesIO()
savez_internal(f, [], {'supersenya': np.array(supersenya)}, compress=True)
print(f.seek(0, io.SEEK_CUR))
f.seek(0)

In [ ]:
l = np.load(f, allow_pickle=False)

In [ ]:
list(l.keys())

In [ ]:
facediff = {k: cmp_face(supersenya, f) for k, f in faces.items()}

In [ ]:
vl = {k: cmp_face(supersenya, f) for k, f in verifies.items() if len(f) == 1}
{k: v for k, v in vl.items() if v > 0.5}

In [ ]:
facediff = {}
for k1, k2 in combinations(faces.keys(), 2):
  facediff[f'{k1}/{k2}'] = cmp_face(faces[k1][0], faces[k2][0])

In [ ]:
from operator import itemgetter
sorted(facediff.items(), key=itemgetter(1), reverse=True)

In [ ]:
for f in [10, 12, 2]:
  print(f, np.linalg.norm(faces[f'photo_{f}'][0] - faces['photo_24'][0]))

In [ ]:
for f in [6, 23]:
  print(f, np.linalg.norm(faces[f'photo_{f}'][0] - faces['photo_2'][0]))
  print(f, np.linalg.norm(faces[f'photo_{f}'][0] - faces['photo_24'][0]))

In [ ]:
!pip install progressbar2

In [ ]:
import progressbar
import os
from random import randint

In [ ]:
from collections import Counter
import operator

In [ ]:
N = 5749
mins = []

with progressbar.ProgressBar(widgets=[
  ' ', progressbar.Counter(), '/%d' % N,
  ' ', progressbar.AdaptiveETA(),
  ], max_value=N) as bar:
  # for i, fn in enumerate(glob.iglob('lfw/*')):
  for i, dirname in enumerate(os.scandir('lfw')):
    # if randint(0, 2) > 0:
    #   continue
    for d in os.scandir(dirname.path):
      fn = d.path

      nam = fn.rsplit('/', 1)[1]
      pic = face_recognition.load_image_file(fn)
      # loc = face_recognition.face_locations(pic, model='cnn')
      loc = face_recognition.face_locations(pic, number_of_times_to_upsample=0, model='cnn')
      if len(loc) == 0:
        continue

      encs = face_recognition.face_encodings(pic, known_face_locations=loc)

      for j, enc in enumerate(encs):
        if same_face(supersenya, enc, 0.65):
          cmps = [cmp_face(x, enc) for x in supersenya]
          mi, mv = min(enumerate(cmps), key=operator.itemgetter(1))
          print(f'{i} same face: {nam}#{j}:\n   {mi} {mv} {cmps}')
          mins.append(mi)
      
      break
    bar.update(i)

len(mins), Counter(mins)

In [ ]:
verifies = {}

for fn in glob.iglob('drive/MyDrive/senya/verify/*'):
  im = PIL.Image.open(fn).convert('RGB')
  a = np.array(im)
  nam = fn.rsplit('/', 1)[1]
  loc = face_recognition.face_locations(a, number_of_times_to_upsample=0, model='cnn')
  enc = face_recognition.face_encodings(a, known_face_locations=loc)
  verifies[nam] = enc

In [ ]:
for nam, enc in verifies.items():
  has_senya = False
  for e in enc:
    if same_face(faces['photo_2'], e, 0.59) or same_face(faces['photo_24'], e, 0.59):
      has_senya = True
  if not has_senya:
    print(f'{nam}: got {len(loc)} faces but no senya')

In [ ]:
for nam, enc in verifies.items():
  has_senya = False
  for e in enc:
    if same_face(supersenya, e, 0.5):
      has_senya = True
  if not has_senya:
    print(f'{nam}: got {len(loc)} faces but no senya')

In [ ]:
fn = 'lfw/Paul_ONeill/Paul_ONeill_0003.jpg'
im = PIL.Image.open(fn).convert('RGB')
print(face_recognition.face_locations(np.array(im), number_of_times_to_upsample=1, model='cnn'))
im

In [ ]:
from itertools import combinations
from IPython import display
import PIL.Image

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
for fn in ['photo_2021-05-25 05.55.56.jpeg', 'photo_2021-05-25 05.56.00.jpeg', 'photo_2021-05-25 06.03.05.jpeg', 'photo_2021-05-25 06.03.07.jpeg', 'photo_2021-05-25 06.06.39.jpeg']:
  im = PIL.Image.open(fn).convert('RGB')
  a = np.array(im)
  nam = fn.rsplit('/', 1)[-1]
  loc = face_recognition.face_locations(a, number_of_times_to_upsample=0, model='cnn')
  enc = face_recognition.face_encodings(a, known_face_locations=loc)
  has_senya = False
  for e in enc:
    # if same_face(faces['photo_2'], e, 0.68) or same_face(faces['photo_24'], e, 0.68):
    if same_face(supersenya, e, 0.69):
      has_senya = True
  if not has_senya:
    print(f'{nam}: got {len(loc)} faces but no senya')
  else:
    print(f"{nam} has senya: {cmp_face(supersenya, e)}")
    display.display(im.resize((s//3 for s in im.size)))


In [ ]:
try:
    import dlib.cuda
    try:
        dlib.cuda.get_num_devices()
        has_cuda = True
    except RuntimeError:
        has_cuda = False
    del dlib.cuda
except ImportError:
    has_cuda = False

In [ ]:
has_cuda